In [3]:
import os

url = os.getenv('QPILOT_BASE_URL')
key = os.getenv('QPILOT_API_KEY')

from langchain.chat_models import ChatOpenAI

llm = OpenAI(model_name="gpt-3.5-turbo", openai_api_key=key, openai_api_base=url)

llm("What day comes after Friday?")

AuthenticationError: Incorrect API key provided: ZXlKaGJH************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************QUU4. You can find your API key at https://platform.openai.com/account/api-keys.

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=1, openai_api_key=key, openai_api_base=url)
chat(
    [
        SystemMessage(content="You are an expert on large language models and can answer any questions related to large language models."),
        HumanMessage(content="What’s the difference between Generic Language Models, Instruction Tuned Models and Dialog Tuned Models")
    ]
)

AuthenticationError: Incorrect API key provided: ZXlKaGJH************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************QUU4. You can find your API key at https://platform.openai.com/account/api-keys.

In [6]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=key, openai_api_base=url)
text_embedding = embeddings.embed_query("To embed text(it can have any length)")
print (f"Your embedding's length: {len(text_embedding)}")
print (f"Here's a sample: {text_embedding[:5]}...")

AuthenticationError: Incorrect API key provided: ZXlKaGJH************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************QUU4. You can find your API key at https://platform.openai.com/account/api-keys.

In [38]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=0.5, model_name="gpt-3.5-turbo", openai_api_key=key, openai_api_base=url)
template = """
## Input
{text}

## Instruction
Please summarize the piece of text in the input part above.
Respond in a manner that a 5 year old would understand.

{format_instructions}

YOUR RESPONSE:
"""

# 创建一个Output Parser，包含两个输出字段，并指定类型和说明
output_parser = StructuredOutputParser.from_response_schemas(
    [
        ResponseSchema(name="keywords", type="list", description="keywords of the text"),
        ResponseSchema(name="summary", type="string", description="summary of the text"),
    ]
)
# 创建Prompt Template，并将format_instructions通过partial_variables直接指定为Output Parser的format
prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)
# 创建Chain并绑定Prompt Template和Output Parser(它将自动使用Output Parser解析llm输出)
summarize_chain = LLMChain(llm=llm, verbose=True, prompt=prompt, output_parser=output_parser)

to_summarize_text = 'Abstract. Text-to-SQL aims at generating SQL queries for the given natural language questions and thus helping users to query databases. Prompt learning with large language models (LLMs) has emerged as a recent approach, which designs prompts to lead LLMs to understand the input question and generate the corresponding SQL. However, it faces challenges with strict SQL syntax requirements. Existing work prompts the LLMs with a list of demonstration examples (i.e. question-SQL pairs) to generate SQL, but the fixed prompts can hardly handle the scenario where the semantic gap between the retrieved demonstration and the input question is large.'
output = summarize_chain.predict(text=to_summarize_text)

import json
print (json.dumps(output, indent=4))



> Entering new  chain...
Prompt after formatting:

## Input
Abstract. Text-to-SQL aims at generating SQL queries for the given natural language questions and thus helping users to query databases. Prompt learning with large language models (LLMs) has emerged as a recent approach, which designs prompts to lead LLMs to understand the input question and generate the corresponding SQL. However, it faces challenges with strict SQL syntax requirements. Existing work prompts the LLMs with a list of demonstration examples (i.e. question-SQL pairs) to generate SQL, but the fixed prompts can hardly handle the scenario where the semantic gap between the retrieved demonstration and the input question is large.

## Instruction
Please summarize the piece of text in the input part above.
Respond in a manner that a 5 year old would understand.

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"keywords

In [2]:
from langchain.document_loaders import NotionDBLoader
from getpass import getpass

# getpass()指引用户输入密钥
NOTION_TOKEN = getpass()
DATABASE_ID = getpass()

loader = NotionDBLoader(
    integration_token=NOTION_TOKEN,
    database_id=DATABASE_ID,
    request_timeout_sec=30,  # optional, defaults to 10
)
# 请求详情见 https://developers.notion.com/reference/post-database-query
docs = loader.load()
docs[0].page_content[:100]

 ········
 ········


HTTPError: 401 Client Error: Unauthorized for url: https://api.notion.com/v1/databases/test/query

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 初始化LLM
llm = ChatOpenAI(temperature=0.5, model_name="gpt-3.5-turbo", openai_api_key=key, openai_api_base=url)

# 加载文档
loader = TextLoader('start.md')
doc = loader.load()
print (f"You have {len(doc)} document")

print (f"You have {len(doc[0].page_content)} characters in that document")

# 分割字符串
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=400)
docs = text_splitter.split_documents(doc)
num_total_characters = sum([len(x.page_content) for x in docs])
print (f"Now you have {len(docs)} documents that have an average of {num_total_characters / len(docs):,.0f} characters (smaller pieces)")

# 初始化向量化模型
embeddings = OpenAIEmbeddings(openai_api_key=key, openai_api_base=url)

# 向量化Document并存入向量数据库（绑定向量和对应Document元数据），这里我们选择本地最常用的FAISS数据库
# 注意: 这会向OpenAI产生请求并产生费用
doc_search = FAISS.from_documents(docs, embeddings)

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=doc_search.as_retriever(), verbose=True)
query = "Specific questions to be asked"
qa.run(query)

You have 1 document
You have 2035 characters in that document
Now you have 1 documents that have an average of 2,028 characters (smaller pieces)


AuthenticationError: Incorrect API key provided: ZXlKaGJH************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************QUU4. You can find your API key at https://platform.openai.com/account/api-keys.